In [1]:
import random
import oandapyV20
from oandapyV20 import API
from oandapyV20.exceptions import V20Error
from oandapyV20.endpoints.instruments import InstrumentsCandles

def get_historical_prices(instrument, count):
    access_token = "e11aadc917842adf254cd73c038c4e0a-321ea21ac5697ab46036807f5e5e943d"

    api = API(access_token=access_token, environment="practice")
    params = {
        "granularity": "H2",
        "count": count,
    }
    request = InstrumentsCandles(instrument=instrument, params=params)
    
    try:
        response = api.request(request)
        candles = response["candles"]
        prices = [float(candle["mid"]["c"]) for candle in candles]
        volumes = [int(candle["volume"]) for candle in candles]
        return prices, volumes
    except V20Error as e:
        print("Fehler beim Abrufen der historischen Preise:", e)
        return [], []

def monte_carlo_simulation(prices, volumes, target_growth, target_decline):
    num_simulations = 1000
    time_to_event = []
    
    fastest_fund = None
    fastest_time = float('inf')
    fastest_event = None
    
    for _ in range(num_simulations):
        simulated_returns = random.choices(prices, k=len(prices))
        simulated_volumes = random.choices(volumes, k=len(volumes))
        fund_value = 1.0
        time = 0
        
        while fund_value < 1 + target_growth and fund_value > 1 - target_decline:
            if time >= len(simulated_returns):
                break
            growth_factor = simulated_returns[time]
            volume_factor = simulated_volumes[time]
            fund_value *= growth_factor
            fund_value *= volume_factor
            time += 1
        
        time_to_event.append(time)
        
        if time < fastest_time:
            fastest_time = time
            fastest_fund = prices[0]  # Erster Wert der Preise verwenden
            if fund_value >= 1 + target_growth:
                fastest_event = "+2% erreicht"
            else:
                fastest_event = "-2% erreicht"
    
    return time_to_event, fastest_fund, fastest_time, fastest_event

# Tickers der Indexfonds
tickers = ['GBP_USD', 'GBP_JPY', 'EUR_CHF', 'AUD_NZD', 'EUR_ZAR', 'NZD_SGD', 'USD_MXN']

# Monte-Carlo-Simulation durchführen
target_growth = 0.02  # 2% Wachstumsziel
target_decline = 0.02  # 2% Rückgangsziel
count = 5000  # Anzahl der zurückliegenden Werte

for ticker in tickers:
    prices, volumes = get_historical_prices(ticker, count)
    
    if len(prices) == 0 or len(volumes) == 0:
        print("Fehler beim Abrufen der historischen Preise für Ticker:", ticker)
        continue
    
    time_to_event, fastest_fund, fastest_time, fastest_event = monte_carlo_simulation(prices, volumes, target_growth, target_decline)
    
    # Ergebnisse anzeigen
    print("Ticker:", ticker)
    print("Durchschnittliche Zeit bis zum Event:", sum(time_to_event) / len(time_to_event))
    print("Schnellster Fund:", fastest_fund)
    print("Zeit bis zum Event:", fastest_time)
    print("Event beim schnellsten Fund:", fastest_event)
    print()



Ticker: GBP_USD
Durchschnittliche Zeit bis zum Event: 1.0
Schnellster Fund: 1.35878
Zeit bis zum Event: 1
Event beim schnellsten Fund: +2% erreicht

Ticker: GBP_JPY
Durchschnittliche Zeit bis zum Event: 1.0
Schnellster Fund: 154.33
Zeit bis zum Event: 1
Event beim schnellsten Fund: +2% erreicht

Ticker: EUR_CHF
Durchschnittliche Zeit bis zum Event: 1.0
Schnellster Fund: 1.07332
Zeit bis zum Event: 1
Event beim schnellsten Fund: +2% erreicht

Ticker: AUD_NZD
Durchschnittliche Zeit bis zum Event: 1.0
Schnellster Fund: 1.06042
Zeit bis zum Event: 1
Event beim schnellsten Fund: +2% erreicht

Ticker: EUR_ZAR
Durchschnittliche Zeit bis zum Event: 1.0
Schnellster Fund: 17.26305
Zeit bis zum Event: 1
Event beim schnellsten Fund: +2% erreicht

Ticker: NZD_SGD
Durchschnittliche Zeit bis zum Event: 1.0
Schnellster Fund: 0.94045
Zeit bis zum Event: 1
Event beim schnellsten Fund: +2% erreicht

Ticker: USD_MXN
Durchschnittliche Zeit bis zum Event: 1.0
Schnellster Fund: 20.74485
Zeit bis zum Event: 1

In [2]:
import random
import oandapyV20
from oandapyV20 import API
from oandapyV20.exceptions import V20Error
from oandapyV20.endpoints.instruments import InstrumentsCandles

def get_historical_prices(instrument, count):
    access_token = "e11aadc917842adf254cd73c038c4e0a-321ea21ac5697ab46036807f5e5e943d"
    
    api = API(access_token=access_token, environment="practice")
    params = {
        "granularity": "D",
        "count": count,
    }
    request = InstrumentsCandles(instrument=instrument, params=params)
    
    try:
        response = api.request(request)
        candles = response["candles"]
        prices = [float(candle["mid"]["c"]) for candle in candles]
        return prices
    except V20Error as e:
        print("Fehler beim Abrufen der historischen Preise:", e)
        return []

def monte_carlo_simulation(prices, target_decline):
    num_simulations = 1000
    time_to_event = []
    fastest_fund = None
    fastest_time = float('inf')
    
    for _ in range(num_simulations):
        simulated_returns = random.choices(prices, k=len(prices))
        fund_value = 1.0
        time = 0
        
        while True:
            if time >= len(simulated_returns):
                break
            growth_factor = simulated_returns[time]
            fund_value *= growth_factor
            time += 1
            
            if fund_value <= 1 - target_decline:
                time_to_event.append(time)
                if time < fastest_time:
                    fastest_time = time
                    fastest_fund = prices[0]  # Erster Wert der Preise verwenden
                break
    
    return time_to_event, fastest_fund, fastest_time

# Tickers der Indexfonds
tickers = ['GBP_USD', 'GBP_JPY', 'EUR_CHF', 'AUD_NZD', 'EUR_ZAR', 'NZD_SGD', 'USD_MXN']
# Monte-Carlo-Simulation durchführen
target_decline = 0.02  # 2% Rückgangsziel
count = 5000  # Anzahl der zurückliegenden Werte

# ...

for ticker in tickers:
    prices = get_historical_prices(ticker, count)
    
    if len(prices) == 0:
        print("Fehler beim Abrufen der historischen Preise für Ticker:", ticker)
        continue
    
    time_to_event, fastest_fund, fastest_time = monte_carlo_simulation(prices, target_decline)
    
    # Ergebnisse anzeigen
    print("Ticker:", ticker)
    
    if len(time_to_event) > 0:
        print("Durchschnittliche Zeit bis zum Event:", sum(time_to_event) / len(time_to_event))
        print("Median der Zeit bis zum Event:", sorted(time_to_event)[len(time_to_event) // 2])
        
        if fastest_fund is not None:
            print("Der schnellste Fonds, der um 2% oder mehr fällt:")
            print("Fonds:", fastest_fund)
            print("Zeit bis zum Event:", fastest_time)
        else:
            print("Kein Fonds erreichte das Rückgangsziel von 2%.")
    else:
        print("Kein Fonds erreichte das Rückgangsziel von 2%.")



Ticker: GBP_USD
Kein Fonds erreichte das Rückgangsziel von 2%.
Ticker: GBP_JPY
Kein Fonds erreichte das Rückgangsziel von 2%.
Ticker: EUR_CHF
Durchschnittliche Zeit bis zum Event: 1.0
Median der Zeit bis zum Event: 1
Der schnellste Fonds, der um 2% oder mehr fällt:
Fonds: 1.5728
Zeit bis zum Event: 1
Ticker: AUD_NZD
Kein Fonds erreichte das Rückgangsziel von 2%.
Ticker: EUR_ZAR
Kein Fonds erreichte das Rückgangsziel von 2%.
Ticker: NZD_SGD
Durchschnittliche Zeit bis zum Event: 2.001
Median der Zeit bis zum Event: 1
Der schnellste Fonds, der um 2% oder mehr fällt:
Fonds: 1.04845
Zeit bis zum Event: 1
Ticker: USD_MXN
Kein Fonds erreichte das Rückgangsziel von 2%.
